In [37]:
import re
import os
import tkinter as tk
from tkinter import filedialog

respond = []
sub = []
#instructions
operation = ['add','sub','lw','sw','addi','addu','addiu','sll','srl','and','or','nor',
             'beq','bne','slt','slti','sltu','j','jal','jr']
#zreo,for IType const,arguments,temp,contents for later,temp,OS,...
reg = {'$zero': 0,
        '$at':0,
        '$v0':0,'$v1':0,
        '$a0':0,'$a1':0,'$a2':0,'$a3':0,
        '$t0':0,'$t1':0,'$t2':0,'$t3':0,'$t4':0,'$t5':0,'$t6':0,'$t7':0,
        '$s0':0,'$s1':0,'$s2':0,'$s3':0,'$s4':0,'$s5':0,'$s6':0,'$s7':0,
        '$t8':0,'$t9':0,
        '$k0':0,'$k1':0,
        '$gp':0,'$sp':0,'$fp':0,'$ra':0 }
reg_table = {'$0':'$zero',
            '$1':'$at',
            '$2':'$v0','$3':'$v1',
            '$4':'$a0','$5':'$a1','$6':'$a2','$7':'$a3',
            '$8':'$t0','$9':'$t1','$10':'$t2','$11':'$t3','$12':'$t4','$13':'$t5','$14':'$t6','$15':'$t7',
            '$16':'$s0','$17':'$s1','$18':'$s2','$19':'$s3','$20':'$s4','$21':'$s5','$22':'$s6','$23':'$s7',
            '$24':'$t8','$25':'$t9',
            '$26':'$k0','$27':'$k1',
            '$28':'$gp','$29':'$sp','$30':'$fp','$31':'$ra'}
PC = 0
insMemOffset = 0
insMem = []
dataMem = {}
identifier = {}
tell = 0



In [38]:
def isNum(c):
    try:    # 如果能运⾏ float(s) 语句，返回 True（字符串 s 是浮点数）        
        float(c)        
        return True    
    except ValueError:  # ValueError 为 Python 的⼀种标准异常，表⽰"传⼊⽆效的参数"        
        pass  # 如果引发了 ValueError 这种异常，不做任何事情（pass：不做任何事情，⼀般⽤做占位语句）    
    try:        
        import unicodedata  # 处理 ASCII 码的包        
        unicodedata.numeric(c)  # 把⼀个表⽰数字的字符串转换为浮点数返回的函数        
        return True    
    except (TypeError, ValueError):        
        pass    
        return False

In [39]:
# trans to binary
def toBin(token,bits): 
    if int(token) < 0 : 
        biNum = bin(abs(int(token))-1)[2:]
    else:
        biNum = bin(abs(int(token)))[2:]
    strbiNum = str(biNum)
    strbiNum = strbiNum.zfill(bits)
    
    if int(token) < 0 : #complement
        nstrbiNum = ""
        for bit in strbiNum:
            if bit =="1":
                nstrbiNum += "0"
            else:
                nstrbiNum += "1"
        print(nstrbiNum)
        return nstrbiNum

    print(strbiNum)
    return strbiNum

In [40]:
# ${letter}->{num},for binary instrction
def transReg(token):
    reg = f"${token}"
    if reg in reg_table: #$2
        reg = token      #reg = 2
    elif reg in reg_table.values():   
        for key,v in reg_table.items():
            if v == reg:
                reg = key.strip( '$' )
                break
    else:
        reg = token
    return reg

In [41]:
def parse(User_input):           #split and classify line to op or reg
    tokens=[]
    splitedLine = re.split('[#:(,\s]+',User_input)
    for c in splitedLine:
        if c != "":
            print(c)
            if c.lower() in operation:
                tokens.append([c,"OPRT"])
            elif isNum(c):
                c = int(c)
                tokens.append([c,"NUM"])
            elif c in reg_table or c in reg_table.values():
                c = c.strip( '$' )
                tokens.append([c,"REG"])
            else:
                tokens.append([c,"Unknown"])
    return tokens

In [42]:
def executeInstr(tokens):
    global respond,identifier,tell,dataMem
    respond = []
    if tokens==False:return False
    
    if (tokens[0][1]=="Unknown"):
        identifier[tokens[0][0]] = tell
        tokens.pop(0)
        
    if(tokens[0][1]=="OPRT"):
        if len(tokens) >= 4:
            if(tokens[0][0].lower() == "add"):
                add(tokens)
            elif(tokens[0][0].lower() == "sub"):
                sub(tokens)
            elif(tokens[0][0].lower() == "lw"):
                lw(tokens)
            elif(tokens[0][0].lower() == "sw"):
                sw(tokens)
            elif(tokens[0][0].lower() == "addi"):
                addi(tokens)
            elif(tokens[0][0].lower() == "addu"):
                addu(tokens)
            elif(tokens[0][0].lower() == "addiu"):
                addiu(tokens)
            elif(tokens[0][0].lower() == "sll" or tokens[0][0].lower() == "srl"):
                shift(tokens)
            elif(tokens[0][0].lower() == "and" or tokens[0][0].lower() == "or" or tokens[0][0].lower() == "nor"):
                AndOrNor(tokens)
            elif(tokens[0][0].lower() == "beq" or tokens[0][0].lower() == "bne"):
                branch(tokens)
            elif(tokens[0][0].lower() == "slt"):
                slt(tokens)
            elif(tokens[0][0].lower() == "slti"):
                slti(tokens)
            elif(tokens[0][0].lower() == "sltu"):
                sltu(tokens)
            else:
                respond.append(["Error",f"Expected identifier."])
        elif len(tokens) == 2:
            if(tokens[0][0].lower() == "j" or tokens[0][0].lower() == "jal" or tokens[0][0].lower() == "jr"):
                jump(tokens)
            else:
                respond.append(["Error",f"Expected identifier."])
        else:
            respond.append(["Error",f"Expected identifier."])
    else:
        respond.append(["Error",f"Unexpected Operation：{tokens[0][0]}."]) 

In [43]:
def lw(tokens):
    # lw $s1,4($t0)
    global respond
    stopExe = False
    # bug detect and response
    for i in range(1,4,1):
        if (i%2==1)and(tokens[i][1] != "REG"):
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        elif i==2 and tokens[i][1] != "NUM":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} must be a number."])
            stopExe = True
        elif(tokens[i][1] == "NUM")and(tokens[i][0] > 65535 or tokens[i][0] < 0):
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is out of range."])
            stopExe = True
        elif (tokens[i][1] == "Unknown"):
            respond.append(["Error",f"Unknown identifier {tokens[i][0]} ."])
    respond.append(["operation",tokens[0][0]])
    
    # execute
    if stopExe == False:
        src1 = f"${tokens[3][0]}"
        src2 = f"${tokens[1][0]}"
        offset = tokens[2][0]
        
        address=toBin(reg[src1]+offset-1,32) 
        data = dataMem.get(address)
        reg[src2] = data
        Itype("100011",tokens[3][0],tokens[1][0],tokens[2][0],1)

def sw(tokens):
    # sw $t1 4($t0)
    global respond,dataMem
    stopExe = False
    # bug detect
    for i in range(1,4,1):   
        if (i%2==1)and(tokens[i][1] != "REG"):
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        elif i==2 and tokens[i][1] != "NUM":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} must be a number."])
            stopExe = True
        elif(tokens[i][1] == "NUM")and(tokens[i][0] > 65535 or tokens[i][0] < 0):
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is out of range."])
            stopExe = True
        elif (tokens[i][1] == "Unknown"):
            respond.append(["Error",f"Unknown identifier {tokens[i][0]} ."])
    respond.append(["operation",tokens[0][0]])
    # execute
    if stopExe == False:
#         respond.append(["source 1",f"${tokens[3][0]}"])
#         respond.append(["source 2",f"${tokens[1][0]}"])
#         address = f"MEM[{tokens[3][0]}+{tokens[2][0]}]"
#         respond.append(["memory address",address])
        src1 = f"${tokens[3][0]}"
        src2 = f"${tokens[1][0]}"
        offset = tokens[2][0]
        for i in range(0,offset,1):
            address=toBin(reg[src1]+i,32) 
            if i != offset-1:
                data = dataMem.get(address)
                if data == None:
                    data = 0
            else:
                data = reg[src2]
            dataMem[address] = data
        Itype("101011",tokens[3][0],tokens[1][0],tokens[2][0],1)

In [44]:
def sub(tokens):
    #sub $s0 $s0 $t1
    global respond
    stopExe = False
    # bug detect
    for i in range(1,4,1): 
        if tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
    # execute
    if stopExe == False:
        src1 = f"${tokens[2][0]}"
        src2 = f"${tokens[3][0]}"
        dest = f"${tokens[1][0]}"
        reg[dest] = reg[src1] - reg[src2]
        Rtype("000000",tokens[2][0],tokens[3][0],tokens[1][0],"00000","100010")
#         respond.append(["destination",f"${tokens[1][0]}"])
#         respond.append(["source 1",f"${tokens[2][0]}"])
#         respond.append(["source 2",f"${tokens[3][0]}"])
       
        
def add(tokens):
    # add $s0 $s0 $t1
    global respond
    stopExe = False
    
    #bug detect
    for i in range(1,4,1):
        if tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
            
    respond.append(["operation",tokens[0][0]])
    
    # execute
    if stopExe == False:
        src1 = f"${tokens[2][0]}"
        src2 = f"${tokens[3][0]}"
        dest = f"${tokens[1][0]}"
        reg[dest] = reg[src1] + reg[src2]
#         respond.append(["destination",f"${tokens[1][0]}"])
#         respond.append(["source 1",f"${tokens[2][0]}"])
#         respond.append(["source 2",f"${tokens[3][0]}"])
        Rtype("000000",tokens[2][0],tokens[3][0],tokens[1][0],"00000","100000")
        
    
def addi(tokens):
    # add $s0 $s0 4
    global respond
    stopExe = False
    
    #bug detect
    for i in range(1,4,1):
        if (i<3) and tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        if (i==3) and tokens[i][1] != "NUM":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is not an immediate value."])
            stopExe = True

    if(tokens[3][1] == "NUM")and(tokens[3][0] > 32767 or tokens[3][0] < -32768):
        respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[3][0]} is out of range."])
        stopExe = True 
        
    respond.append(["operation",tokens[0][0]])
    
    # execute
    if stopExe == False:
        src1 = f"${tokens[2][0]}"
        val = tokens[3][0]
        dest = f"${tokens[1][0]}"
        reg[dest] = reg[src1] + int(val)
#         respond.append(["destination",f"${tokens[1][0]}"])
#         respond.append(["source 1",f"${tokens[2][0]}"])
#         respond.append(["immediate value",tokens[3][0]])
        Itype("001000",tokens[2][0],tokens[1][0],tokens[3][0],1)
            
def addu(tokens):
    # addu $s0 $s0 $t1
    global respond
    stopExe = False
    
    # bug detect
    for i in range(1,4,1):
        if tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
            
    #check unsign
    if stopExe == False:
        register = f"${tokens[3][0]}"
        if register in reg_table:
            rt = reg_table[register]
        else:
            rt = register
        value = reg[rt]
        if value != None and (value > 65535 or value < 0):
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[3][0]} is out of range."])
            stopExe = True 
    
    respond.append(["operation",tokens[0][0]])
    # execution
    if stopExe == False:
        src1 = f"${tokens[2][0]}"
        src2 = f"${tokens[3][0]}"
        dest = f"${tokens[1][0]}"
        reg[dest] = reg[src1] + reg[src2]
#         respond.append(["destination",f"${tokens[1][0]}"])
#         respond.append(["source 1",f"${tokens[2][0]}"])
#         respond.append(["source 2",f"${tokens[3][0]}"])
        Rtype("000000",tokens[2][0],tokens[3][0],tokens[1][0],"00000","100001")
            
def addiu(tokens):
    # addiu $s0 $s0 4
    global respond
    stopExe = False

    # bug detect
    for i in range(1,4,1):
        if (i<3) and tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        if (i==3) and tokens[i][1] != "NUM":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is not an immediate value."])
            stopExe = True
    
    #check unsign
    if(tokens[3][1] == "NUM")and(tokens[3][0] > 65535 or tokens[3][0] < 0):
        respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[3][0]} is out of range."])
        stopExe = True 
        
    respond.append(["operation",tokens[0][0]])
    # execute
    if stopExe == False:
        src1 = f"${tokens[2][0]}"
        val = tokens[3][0]
        dest = f"${tokens[1][0]}"
        reg[dest] = reg[src1] + val
#         respond.append(["destination",f"${tokens[1][0]}"])
#         respond.append(["source 1",f"${tokens[2][0]}"])
#         respond.append(["immediate value",tokens[3][0]])
        Itype("001001",tokens[2][0],tokens[1][0],tokens[3][0],1)

In [45]:
def shift(tokens):
    # sll   $t1, $s0,2
    global respond
    stopExe = False
    # bug detect
    for i in range(1,4,1):
        if (i<3) and tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        if (i==3) and tokens[i][1] != "NUM":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is not an immediate value."])
            stopExe = True
            
    if(tokens[3][1] == "NUM")and (tokens[3][0] > 31 or tokens[i][0] < 0):
        respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[3][0]} is out of range."])
        stopExe = True 
        
    respond.append(["operation",tokens[0][0]])
    # execute
    if stopExe == False:
#         respond.append(["destination",f"${tokens[1][0]}"])
#         respond.append(["source 1",f"${tokens[2][0]}"])
#         respond.append(["shift amount",tokens[3][0]])
        src1 = f"${tokens[2][0]}"
        val = tokens[3][0]
        dest = f"${tokens[1][0]}"
        if tokens[0][0].lower() == "sll":
            reg[dest] = reg[src1] *2*val
            Rtype("000000","00000",tokens[2][0],tokens[1][0],tokens[3][0],"000000")
        else:
            reg[dest] = reg[src1] /(2*val)
            Rtype("000000","00000",tokens[2][0],tokens[1][0],tokens[3][0],"000010")

In [46]:
def AndOrNor(tokens):
    # AND $t3, $t2, $t1
    global respond
    stopExe = False
    #bug detect
    for i in range(1,4,1):
        if tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
            
    respond.append(["operation",tokens[0][0]])
    #execute
    if stopExe == False:
#         respond.append(["destination",f"${tokens[1][0]}"])
#         respond.append(["source 1",f"${tokens[2][0]}"])
#         respond.append(["source 2",f"${tokens[3][0]}"])
        src1 = f"${tokens[2][0]}"
        src2 = f"${tokens[3][0]}"
        dest = f"${tokens[1][0]}"
        if tokens[0][0].lower() == "and":
            reg[dest] = reg[src1] & reg[src2]
            Rtype("000000",tokens[2][0],tokens[1][0],tokens[3][0],"00000","100100")
        elif tokens[0][0].lower() == "or":
            reg[dest] = reg[src1] | reg[src2]
            Rtype("000000",tokens[2][0],tokens[1][0],tokens[3][0],"00000","100101")
        else:
            reg[dest] = (reg[src1] | reg[src2])^0b11111111111111111111111111111111
            Rtype("000000",tokens[2][0],tokens[1][0],tokens[3][0],"00000","100111")

In [47]:
def slt(tokens):
    # slt $t0 $t1 $s0
    global respond
    stopExe = False
    # bug detect
    for i in range(1,4,1):
        if tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
            
    respond.append(["operation",tokens[0][0]])
    # execute
    if stopExe == False:
#         respond.append(["destination",f"${tokens[1][0]}"])
#         respond.append(["source 1",f"${tokens[2][0]}"])
#         respond.append(["source 2",f"${tokens[3][0]}"])
        src1 = f"${tokens[2][0]}"
        src2 = f"${tokens[3][0]}"
        dest = f"${tokens[1][0]}"
        if reg[src1] < reg[src2]:
            reg[dest] = 1
        else:
            reg[dest] = 0
        Rtype("000000",tokens[2][0],tokens[3][0],tokens[1][0],"00000","101010")
        
def slti(tokens):
    # slt $t0 $t1 3
    global respond
    stopExe = False
    # bug detect
    for i in range(1,4,1):
        if (i<3) and tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
        if (i==3) and tokens[i][1] != "NUM":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is not an immediate value."])
            stopExe = True
            
    if(tokens[3][1] == "NUM")and(tokens[3][0] > 32767 or tokens[3][0] < -32768):
        respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[3][0]} is out of range."])
        stopExe = True 
        
    respond.append(["operation",tokens[0][0]])
    # execute
    if stopExe == False:
#         respond.append(["destination",f"${tokens[1][0]}"])
#         respond.append(["source 1",f"${tokens[2][0]}"])
#         respond.append(["immediate value",tokens[3][0]])
        src1 = f"${tokens[2][0]}"
        val = tokens[3][0]
        dest = f"${tokens[1][0]}"
        if reg[src1] < val:
            reg[dest] = 1
        else:
            reg[dest] = 0
        Itype("001010",tokens[2][0],tokens[1][0],tokens[3][0],1)
        
def sltu(tokens):
    global respond
    stopExe = False
    #bug detect
    for i in range(1,4,1):
        if tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
            
    #check unsign
    if stopExe == False:
        register = f"${tokens[3][0]}"
        if register in reg_table:
            rt = reg_table[register]
        else:
            rt = register
        value = reg[rt]
        if value != None and (value > 65535 or value < 0):
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[3][0]} is out of range."])
            stopExe = True 
    
    respond.append(["operation",tokens[0][0]])
    #execute
    if stopExe == False:
#         respond.append(["destination",f"${tokens[1][0]}"])
#         respond.append(["source 1",f"${tokens[2][0]}"])
#         respond.append(["source 2",f"${tokens[3][0]}"])
        src1 = f"${tokens[2][0]}"
        src2 = f"${tokens[3][0]}"
        dest = f"${tokens[1][0]}"
        if reg[src1] < reg[src2]:
            reg[dest] = 1
        else:
            reg[dest] = 0
        Rtype("000000",tokens[2][0],tokens[3][0],tokens[1][0],"00000","101011")

In [48]:
def branch(tokens):
    global respond,identifier
    stopExe = False
    # bug detect
    for i in range(1,4,1):
        if i<3 and tokens[i][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, no suitable register {tokens[i][0]}."])
            stopExe = True
    if tokens[3][1] == "REG":
        respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} can not be a label."])
        stopExe = True
    elif tokens[3][1] == "NUM" and (tokens[3][0] > 32767 or tokens[3][0] < -32768):
        respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is out of range"])
        stopExe = True
    elif tokens[3][1] == "NUM" and tokens[3][0]%4 != 0:
        respond.append(["Waring",f"Malformed {tokens[0][0]} statement, {tokens[i][0]} is not an address."])

            
    respond.append(["operation",tokens[0][0]])
    # execute
    if stopExe == False:
        src1 = f"${tokens[1][0]}"
        src2 = f"${tokens[2][0]}"
        ident = tokens[3][0]
        oriPtr = f.tell()
        pointer = identifier.get(ident)
        distance = -1
        while pointer == None:
            distance += 1
            pointer = f.tell()
            findID = f.readline()
            findID = ''.join(findID.replace(' ',',',1))
            findID = ''.join(findID.replace(')',' ',1))
            findID = ''.join(findID.replace(':',' ',1))
            subTokens = parse(findID)
            if (subTokens[0][1]=="Unknown"):
                identifier[subTokens[0][0]] = pointer
            pointer = identifier.get(ident)
            
        tokens[3][0] = distance
        if tokens[0][0].lower() == "beq":
            if reg[src1] == reg[src2]:
                f.seek(pointer,0)
            else :f.seek(oriPtr,0)
            Itype("000100",tokens[1][0],tokens[2][0],tokens[3][0],0)
        else:
            if reg[src1] != reg[src2]:
                f.seek(pointer,0)
            else :f.seek(oriPtr,0)
            Itype("000101",tokens[1][0],tokens[2][0],tokens[3][0],0)

In [49]:
def jump(tokens):
    global respond,identifier
    stopExe = False
    # bug detect
    if tokens[0][0].lower() != "jr":
        if tokens[1][1] == "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[1][0]} can not be a label."])
            stopExe = True
        if tokens[1][1] == "NUM" and tokens[1][0] > 67108863 :
            respond.append(["Error",f"Malformed {tokens[0][0]} statement, {tokens[1][0]} is out of range."])
            stopExe = True 
            
    else:
        if tokens[1][1] != "REG":
            respond.append(["Error",f"Malformed {tokens[0][0]} statement,  no suitable register {tokens[1][0]}."])
            stopExe = True
        if tokens[1][0] != "$ra" and tokens[1][0] != "$31":
            respond.append(["Waring",f"Malformed {tokens[0][0]} statement, {tokens[1][0]} is not a return address register."])
    
    respond.append(["operation",tokens[0][0]])
    # execute
    if stopExe == False:
#         respond.append(["address",tokens[1][0]])
        if tokens[0][0].lower() == "j" or tokens[0][0].lower() == "jal":
            ident = tokens[1][0]
            oriPtr = f.tell()
            pointer = identifier.get(ident)
            distance = 0
            while pointer == None:
                distance += 1
                pointer = f.tell()
                findID = f.readline()
                findID = ''.join(findID.replace(' ',',',1))
                findID = ''.join(findID.replace(')',' ',1))
                findID = ''.join(findID.replace(':',' ',1))
                subTokens = parse(findID)
                if (subTokens[0][1]=="Unknown"):
                    identifier[subTokens[0][0]] = pointer
                pointer = identifier.get(ident)
            tokens[1][0] = distance*4 + 1
            
        if tokens[0][0].lower() == "j":
            f.seek(pointer,0)
            Jtype("000010",tokens[1][0])
        elif tokens[0][0].lower() == "jal":
            identifier["jal"] = oriPtr           #link PC+4
            f.seek(pointer,0)                    #jump  
            Jtype("000011",tokens[1][0])
        else:
            # jr
            pointer = identifier["jal"]       
            f.seek(pointer,0)
            Rtype("000000",tokens[1][0],"00000","00000","00000","001000")

In [50]:
def Rtype(op,rs,rt,rd,shamt,funct):
    global insMemOffset,insMem
    rs = toBin(transReg(rs),5)
    rt = toBin(transReg(rt),5)
    rd = toBin(transReg(rd),5)
    shamt = toBin(shamt,5)
    fullIns = op + rs + rt + rd + shamt +funct
    print(f"fullIns: {fullIns}")
    for i in range(0,32,8):
        strOffset = toBin(insMemOffset,32)
        insMem.append([strOffset,fullIns[i:i+8]])
        insMemOffset += 1
    
def Itype(op,rs,rt,imm,flag):
    global insMemOffset,insMem
    rs = toBin(transReg(rs),5)
    rt = toBin(transReg(rt),5)
    imm = toBin(imm,16)
    fullIns = op + rs + rt + imm
    print(f"fullIns: {fullIns}")
    for i in range(0,32,8):
        strOffset = toBin(insMemOffset,32)
        insMem.append([strOffset,fullIns[i:i+8]])
        insMemOffset += 1
        
def Jtype(op,address):
    global insMemOffset,insMem
    address = toBin(address,26)
    respond.append(["opcode",op])
    respond.append(["address",address])

In [51]:
def initial():
    global PC,insMemOffset,insMem,reg
    f.seek(0,0)
    PC = 0
    insMemOffset = 0
    insMem = []
    dataMem = {}
    identifier = {}
    reg = {'$zero': 0,
        '$at':0,
        '$v0':0,'$v1':0,
        '$a0':0,'$a1':0,'$a2':0,'$a3':0,
        '$t0':0,'$t1':0,'$t2':0,'$t3':0,'$t4':0,'$t5':0,'$t6':0,'$t7':0,
        '$s0':0,'$s1':0,'$s2':0,'$s3':0,'$s4':0,'$s5':0,'$s6':0,'$s7':0,
        '$t8':0,'$t9':0,
        '$k0':0,'$k1':0,
        '$gp':0,'$sp':0,'$fp':0,'$ra':0 }

In [52]:
# GUI 
User_input=''
window = tk.Tk()

width = 960
height = 540
window_width = window.winfo_screenwidth()
window_height = window.winfo_screenheight()
left = int((window_width - width)/2)
top = int((window_height - height)/2) 
Str_Len = 30

window.title('MIPS instruction decoder 3') 
window.geometry(f'{width}x{height}+{left}+{top}') # window show size
window.configure(bg='#fff') # window BG color
label = tk.Label(window, bd=10, bg='#ffa',fg='#000', text = "MIPS instruction decoder").pack(side="top",fill="x") # Title Label 

iniBtn = tk.Button(window, width=10, height=1,bg="#ffa", command=lambda:show(0),text='Go to first line', font=("Times",10)) # button  
iniBtn.place(relx=0.4,y=140,anchor = 'center')

nextBtn = tk.Button(window, width=8, height=1,bg="#ffa", command=lambda:show(-1),text='Next Line', font=("Times",10)) # button  
nextBtn.place(relx=0.6,y=140,anchor = 'center')

# chose file & open file
fileName = filedialog.askopenfilename(title='Select file')
f = open(fileName,"r")


In [ ]:
def show(line):
    global User_input,respond,PC,insMemOffset,reg,dataMem,tell
    tell = f.tell()
    if line == 0:
        initial()
    User_input = f.readline()
    raw_input = User_input
    User_input = ''.join(User_input.replace(' ',',',1))
    User_input = ''.join(User_input.replace(')',' ',1))
    User_input = ''.join(User_input.replace(':',' ',1))
#     User_input = User_input.strip(')')
#     print(User_input)
    # decode and execute
    print(parse(User_input))
    executeInstr(parse(User_input)) 
    PC += 4
    binPC = toBin(PC,32)

    # show instrction
    instrc = tk.Label(window,width=20, bg='#fff', text = raw_input, font=("Times",14,"bold"))
    instrc.place(relx=0.45,y=190)
    # show decoded data
    text = tk.Text(window,height=20, font=("Times",14,"bold"))
    # error respond
    if type(respond) != str:
        for res in respond:
            text.insert(tk.INSERT,f"{res[0]}: {res[1]}\n")
    # PC
    text.insert(tk.INSERT,f"PC: {binPC}\n")
    # Register file
    text.insert(tk.INSERT,"Register File:\n")
    text.insert(tk.INSERT,f"\t$t0: {toBin(reg['$t0'],32)}\n")
    text.insert(tk.INSERT,f"\t$t1: {toBin(reg['$t1'],32)}\n")
    text.insert(tk.INSERT,f"\t$s0: {toBin(reg['$s0'],32)}\n")
    text.insert(tk.INSERT,f"\t$s1: {toBin(reg['$s1'],32)}\n")
    # Instruction Memory
    text.insert(tk.INSERT,"Instruction Memory:\n")
    if type(insMem) != str:
        for i in range(insMemOffset-4,insMemOffset,1):
            text.insert(tk.INSERT,f"\tMem[{insMem[i][0]}] = {insMem[i][1]}\n")  
    text.place(relx=0.2,rely=0.5)
    # Data Memory
    text.insert(tk.INSERT,"Data Memory:\n")
    for key,val in dataMem.items():
        print(f"key: {key}")
        print(val)
        text.insert(tk.INSERT,f"\tMem[{key}] = {toBin(val,8)}\n")  
    return respond

window.mainloop() 
print(User_input)

Judgment
beq
$t1
$t0
Initialization
[['Judgment', 'Unknown'], ['beq', 'OPRT'], ['t1', 'REG'], ['t0', 'REG'], ['Initialization', 'Unknown']]
Judgment
beq
$t1
$t0
Initialization
j
Exit
Initialization
addi
$t0
$t0
4
01001
01000
0000000000000001
fullIns: 00010001001010000000000000000001
00000000000000000000000000000000
00000000000000000000000000000001
00000000000000000000000000000010
00000000000000000000000000000011
00000000000000000000000000000100
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
00000000000000000000000000000000
Initialization
addi
$t0
$t0
4
[['Initialization', 'Unknown'], ['addi', 'OPRT'], ['t0', 'REG'], ['t0', 'REG'], [4, 'NUM']]
Initialization
addi
$t0
$t0
4
01000
01000
0000000000000100
fullIns: 00100001000010000000000000000100
00000000000000000000000000000100
00000000000000000000000000000101
00000000000000000000000000000110
00000000000000000000000000000111
00000000000000000000000000001000
0000000000000000000000000000010

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\python3.9.0\lib\tkinter\__init__.py", line 1885, in __call__
    return self.func(*args)
  File "C:\Users\huangxh\AppData\Local\Temp\ipykernel_12064\3314679748.py", line 21, in <lambda>
    nextBtn = tk.Button(window, width=8, height=1,bg="#ffa", command=lambda:show(-1),text='Next Line', font=("Times",10)) # button
  File "C:\Users\huangxh\AppData\Local\Temp\ipykernel_12064\2871794326.py", line 15, in show
    executeInstr(parse(User_input))
  File "C:\Users\huangxh\AppData\Local\Temp\ipykernel_12064\892214466.py", line 6, in executeInstr
    if (tokens[0][1]=="Unknown"):
IndexError: list index out of range


[]


Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\python3.9.0\lib\tkinter\__init__.py", line 1885, in __call__
    return self.func(*args)
  File "C:\Users\huangxh\AppData\Local\Temp\ipykernel_12064\3314679748.py", line 21, in <lambda>
    nextBtn = tk.Button(window, width=8, height=1,bg="#ffa", command=lambda:show(-1),text='Next Line', font=("Times",10)) # button
  File "C:\Users\huangxh\AppData\Local\Temp\ipykernel_12064\2871794326.py", line 15, in show
    executeInstr(parse(User_input))
  File "C:\Users\huangxh\AppData\Local\Temp\ipykernel_12064\892214466.py", line 6, in executeInstr
    if (tokens[0][1]=="Unknown"):
IndexError: list index out of range


[]


Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\python3.9.0\lib\tkinter\__init__.py", line 1885, in __call__
    return self.func(*args)
  File "C:\Users\huangxh\AppData\Local\Temp\ipykernel_12064\3314679748.py", line 21, in <lambda>
    nextBtn = tk.Button(window, width=8, height=1,bg="#ffa", command=lambda:show(-1),text='Next Line', font=("Times",10)) # button
  File "C:\Users\huangxh\AppData\Local\Temp\ipykernel_12064\2871794326.py", line 15, in show
    executeInstr(parse(User_input))
  File "C:\Users\huangxh\AppData\Local\Temp\ipykernel_12064\892214466.py", line 6, in executeInstr
    if (tokens[0][1]=="Unknown"):
IndexError: list index out of range


[]


Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\python3.9.0\lib\tkinter\__init__.py", line 1885, in __call__
    return self.func(*args)
  File "C:\Users\huangxh\AppData\Local\Temp\ipykernel_12064\3314679748.py", line 21, in <lambda>
    nextBtn = tk.Button(window, width=8, height=1,bg="#ffa", command=lambda:show(-1),text='Next Line', font=("Times",10)) # button
  File "C:\Users\huangxh\AppData\Local\Temp\ipykernel_12064\2871794326.py", line 15, in show
    executeInstr(parse(User_input))
  File "C:\Users\huangxh\AppData\Local\Temp\ipykernel_12064\892214466.py", line 6, in executeInstr
    if (tokens[0][1]=="Unknown"):
IndexError: list index out of range


In [ ]:
identifier